In [1]:
from __future__ import unicode_literals,print_function

import json
import plac
import random
import spacy
import warnings
from pathlib import Path
from spacy.util import minibatch,compounding

/home/zijian/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
nlp_de1 = spacy.load("de_core_news_sm")

In [3]:
nlp_de2 = spacy.load("de_core_news_md")

In [4]:
SM_plus_Train_DATA_Model = spacy.load('ModelsRetrained/Novo_Models/SpacySM+TrainData')

In [5]:
MD_plus_Train_DATA_Model = spacy.load('ModelsRetrained/Novo_Models/SpacyMD+TrainData')

In [ ]:
def exemple()

### Create train data

In [6]:
with open("novo_train_de.json",'r',encoding="utf-8")as f:
    train_data = json.load(f)
print(train_data[0])

def traiter_texte(text):
    text = text.replace("\n"," ").replace("\t"," ")
    text = text.replace("Institutin Zürich","Institut in Zürich")
    text = text.replace("desneuen"," des neuen")
    text = text.replace("delas","de las")
    text = text.replace("A. Z.)Belgien","A. Z.) Belgien")
    text = text.replace("derBompartschen","der Bompartschen")
    text = text.replace("aufZürich","auf Zürich")
    text = text.replace("derSociété","der Société")
    return text

def spacy_format_for_train(data):    
    label_map = {
        'pers':'PER',
        'org':'ORG',
        'loc':'LOC',
    } 
    DATA = []
    for i in range(len(data)):
        values = [(x[1],x[2],label_map.get(x[3],"MISC")) for x in data[i][1:]]
        data[i][0] = traiter_texte(data[i][0])
        DATA.append((data[i][0],{"entities":values}))
    return DATA
    
TRAIN_DATA = spacy_format_for_train(train_data)

['Frankreich.', ['Frankreich', 0, 10, 'loc']]


A. Z.)Belgien

desneuen

derBompartschen

aufZürich

Institutin Zürich

derSociété

dem

den

laPlata

delas = de las

### Training data


##### 1. TRAIN_DATA

In [32]:
import random
from random import shuffle

def create_model(output_dir,n_iter,TRAIN_DATA):
    nlp = spacy.blank("de")
    
    # get ner pipelines for this model so that we can modify labels
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner,last=True)
    
    
    # add labels
    for x,y in TRAIN_DATA:
        for ent in y.get("entities"):
            ner.add_label(ent[2])
        
    # train ner but not others
    pipe_exceptions = set(["ner","trf_wordpiecer","trf_tok2vec"])
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    with nlp.disable_pipes(*other_pipes) and warnings.catch_warnings():
        warnings.filterwarnings("once",category=UserWarning,module="spacy")
        nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            
            # batch up the examples using spaCy's minibatch （speed up for training)
            batches = minibatch(TRAIN_DATA,size=compounding(4.0,32.0,1.001))
            for batch in batches:
                texts,annotations = zip(*batch)
                nlp.update(
                    texts,
                    annotations,
                    drop=0.5,
                    losses=losses,
                )
            print("Losses,",losses)
    
    nlp.to_disk(output_dir)
    
create_model("/Users/zijian/ZijianStageEntiteNommee/Detect-Named-Entity-/spacyNER/ModelsRetrained/Novo_Models/Create/",110,TRAIN_DATA)

Losses, {'ner': 22510.204658385643}
Losses, {'ner': 5584.152738349512}
Losses, {'ner': 5205.596113906242}
Losses, {'ner': 4904.141694826772}
Losses, {'ner': 4630.302443271037}
Losses, {'ner': 4499.311127712717}
Losses, {'ner': 4302.91619207905}
Losses, {'ner': 4017.5811725113017}
Losses, {'ner': 3837.705194377282}
Losses, {'ner': 3760.280830729651}
Losses, {'ner': 3447.6288873856465}
Losses, {'ner': 3515.3563000527465}
Losses, {'ner': 3359.4275583038107}
Losses, {'ner': 3331.8696475688357}
Losses, {'ner': 3147.569693643114}
Losses, {'ner': 3135.810705791948}
Losses, {'ner': 2875.1937318446053}
Losses, {'ner': 2953.5157632302667}
Losses, {'ner': 2779.487574584862}
Losses, {'ner': 2721.70334248339}
Losses, {'ner': 2603.708678103254}
Losses, {'ner': 2536.2681775755636}
Losses, {'ner': 2395.7268400312787}
Losses, {'ner': 2453.1335528004347}
Losses, {'ner': 2259.523928835707}
Losses, {'ner': 2304.5079980807964}
Losses, {'ner': 2094.903223476547}
Losses, {'ner': 2438.2418238018618}
Losses, {

##### 2. Update de_core_web_sm Model with Train Data

In [22]:
def training(TRAIN_DATA,model,output_dir,n_iter):
    nlp = model
    
    # get ner pipelines for this model so that we can modify labels
    ner = nlp.get_pipe("ner")
    
    # add labels
    for x,y in TRAIN_DATA:
        for ent in y.get("entities"):
            ner.add_label(ent[2])
        
    # train ner but not others
    pipe_exceptions = set(["ner","trf_wordpiecer","trf_tok2vec"])
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    with nlp.disable_pipes(*other_pipes) and warnings.catch_warnings():
        warnings.filterwarnings("once",category=UserWarning,module="spacy")
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            
            # batch up the examples using spaCy's minibatch （speed up for training)
            batches = minibatch(TRAIN_DATA,size=compounding(4.0,32.0,1.001))
            for batch in batches:
                texts,annotations = zip(*batch)
                nlp.update(
                    texts,
                    annotations,
                    drop=0.5,
                    losses=losses,
                )
            print("Losses ",itn," ",losses)
    
    nlp.to_disk(output_dir)

#training(TRAIN_DATA,nlp_de1,"/Users/zijian/ZijianStageEntiteNommee/Detect-Named-Entity-/spacyNER/ModelsRetrained/Novo_Models/SpacySM+TrainData",110)


##### 3. Update de_core_web_md Model with Train Data

In [23]:
training(TRAIN_DATA,nlp_de2,"/Users/zijian/ZijianStageEntiteNommee/Detect-Named-Entity-/spacyNER/ModelsRetrained/Novo_Models/SpacyMD+TrainData",110)

Losses  0   {'ner': 6873.515508462209}
Losses  1   {'ner': 5420.311381392414}
Losses  2   {'ner': 4881.413963082014}
Losses  3   {'ner': 4949.26579183084}
Losses  4   {'ner': 4540.505032080226}
Losses  5   {'ner': 4649.668678913207}
Losses  6   {'ner': 4179.612263078336}
Losses  7   {'ner': 4144.333322366932}
Losses  8   {'ner': 3951.566984627396}
Losses  9   {'ner': 4165.061035813764}
Losses  10   {'ner': 3782.329937248258}
Losses  11   {'ner': 3940.0934726260602}
Losses  12   {'ner': 4097.769276100793}
Losses  13   {'ner': 3667.679412432597}
Losses  14   {'ner': 3907.7812510203803}
Losses  15   {'ner': 3608.5714578921325}
Losses  16   {'ner': 3594.435505003319}
Losses  17   {'ner': 4106.746974390582}
Losses  18   {'ner': 3406.1804820275865}
Losses  19   {'ner': 3425.69815871818}
Losses  20   {'ner': 3414.984659321548}
Losses  21   {'ner': 3306.9726137538673}
Losses  22   {'ner': 3298.014982529632}
Losses  23   {'ner': 3478.6502737245173}
Losses  24   {'ner': 3151.816351400339}
Losses

### Evaluer 2 nouveaux modèles en utilisant dev data

In [54]:
def context(text,i,j):
    left,right = i,j
    count = 0
    if left > 0:
        while count < 5 and left > 0:
            left-=1
            if text[left]==' ':
                count+=1
    count = 0
    while count < 5 and right+1 < len(text):
        right += 1
        if text[right]==' ':
            count += 1
    return text[left:right]

def consulter(model,dev):
    results = []
    for i in range(len(dev)):
        doc = model(dev[i][0])
        tmp = []
        for ent in doc.ents:
            tmp.append([ent.text,ent.start_char,ent.end_char,ent.label_,context(dev[i][0],ent.start_char,ent.end_char)])
        results.append(tmp) 
    return results

#### Appeler 3 nouveaux modèles entraînés et deux modèles existants

In [24]:
MD_plus_Train_DATA_Model = spacy.load('ModelsRetrained/Novo_Models/SpacyMD+TrainData')

In [33]:
nlp_create = spacy.load('ModelsRetrained/Novo_Models/Create')

In [55]:
with open("novo_dev_de.json",'r',encoding="utf-8")as f:
    dev = json.load(f)
sm = consulter(nlp_de1,dev)
md = consulter(nlp_de2,dev)
sm_train = consulter(SM_plus_Train_DATA_Model,dev)
md_train = consulter(MD_plus_Train_DATA_Model,dev)
create = consulter(nlp_create,dev)
print(len(sm))
print(len(md))
print(len(sm_train))
print(len(md_train))
print(len(create))

40
40
40
40
40


In [30]:
def ecrire(filename,data):
    import json
    with open("résultat/{}".format(filename),"w",encoding="utf-8") as f:
        json.dump(data,f,ensure_ascii=False)
ecrire("spacy sm.json",sm)
ecrire("spacy md.json",md)
ecrire("spacy sm + train.json",sm_train)
ecrire("spacy md + train.json",md_train)

### Gold 

In [13]:
import json
with open("novo_dev_de.json",'r',encoding="utf-8")as f:
    dev = json.load(f)
DEV_DATA = spacy_format_for_train(dev)
for i in range(len(DEV_DATA)):
    DEV_DATA[i] = (DEV_DATA[i][0],DEV_DATA[i][1]["entities"])

### Test

In [14]:
from spacy.scorer import Scorer
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model,data):
    scorer = Scorer()
    for text,annot in data:
        doc_gold_text = ner_model.make_doc(text)
        gold = GoldParse(doc_gold_text,entities=annot)
        pred_value = ner_model(text)
        scorer.score(pred_value,gold)
    return scorer.scores

### create

In [34]:
res_create = evaluate(nlp_create,DEV_DATA)
res_create

{'uas': 0.0,
 'las': 0.0,
 'ents_p': 62.0,
 'ents_r': 46.96969696969697,
 'ents_f': 53.44827586206896,
 'tags_acc': 0.0,
 'token_acc': 100.0}

### sm

In [28]:
res_sm = evaluate(nlp_de1,DEV_DATA)
res_sm

{'uas': 0.0,
 'las': 0.0,
 'ents_p': 33.02752293577982,
 'ents_r': 54.54545454545454,
 'ents_f': 41.14285714285714,
 'tags_acc': 0.0,
 'token_acc': 100.0}

### sm_train

In [16]:
res_sm_train = evaluate(SM_plus_Train_DATA_Model,DEV_DATA)
res_sm_train

{'uas': 0.0,
 'las': 0.0,
 'ents_p': 76.0,
 'ents_r': 57.57575757575758,
 'ents_f': 65.51724137931035,
 'tags_acc': 0.0,
 'token_acc': 100.0}

### md

In [26]:
res_md = evaluate(nlp_de2,DEV_DATA)
res_md

{'uas': 0.0,
 'las': 0.0,
 'ents_p': 68.75,
 'ents_r': 66.66666666666666,
 'ents_f': 67.6923076923077,
 'tags_acc': 0.0,
 'token_acc': 100.0}

### md+train

In [27]:
res_md_train = evaluate(MD_plus_Train_DATA_Model,DEV_DATA)
res_md_train

{'uas': 0.0,
 'las': 0.0,
 'ents_p': 70.6896551724138,
 'ents_r': 62.121212121212125,
 'ents_f': 66.12903225806453,
 'tags_acc': 0.0,
 'token_acc': 100.0}

### Train by updating the model in spacy de_core_web_sm

### So What is the difference? Model 1 could detect what? Model 2 could detect what?

### Référence

In [57]:
def correction():
    correction = []
    for i in range(len(dev)):
        text = dev[i][0]
        tmp = []
        for x in dev[i][1:]:
            tmp.append([x[0],x[1],x[2],x[3],context(text,x[1],x[2])])
        correction.append(tmp[:])
    return correction
reference = correction()
print(reference[1])

[['Petersburg', 11, 21, 'loc', '(Condeer.] Petersburg den 18. Dec. Se.russisch- kaiserl.'], ['Prinzen', 76, 83, 'pers', ' kaiserl. Majestät haben dem Prinzen vonCondé bey dessen Ankunft in'], ['Petersburg', 115, 125, 'loc', ' bey dessen Ankunft in Petersburg den St.Andreas-Orden u. den Maltheser'], ['Polen', 179, 184, 'loc', ' den Maltheser Ritterorden in Polen zu ertheilen, und ihn mit'], ['Petersburg', 256, 266, 'loc', ' prächtigen,völlig meublirten Palais in Petersburg zu beschenken geruhet. Das aus'], ['Prinzen', 362, 369, 'pers', ' Kavallerie-Regimentern bestehende Corps des Prinzen vonCondé, welches in kaiserliche Dienste'], ['Wladimir', 440, 448, 'loc', ' genommenworden, ist nun nach Wladimir, Luzk und Kowelin Quartier'], ['Luzk', 450, 454, 'loc', ' ist nun nach Wladimir, Luzk und Kowelin Quartier verlegt. Das'], ['Kowelin', 459, 466, 'loc', ' nach Wladimir, Luzk und Kowelin Quartier verlegt. Das ganze Corps'], ['Prinzen', 538, 545, 'pers', ' wird unterbestandiger Inspection des

In [61]:
for i in range(len(sm_train[1])):
    print(sm_train[1][i])
    print(reference[1][i])
    print("\n")

['Condeer', 1, 8, 'LOC', '(Condeer.] Petersburg den 18. Dec.']
['Petersburg', 11, 21, 'loc', '(Condeer.] Petersburg den 18. Dec. Se.russisch- kaiserl.']


['Petersburg', 11, 21, 'LOC', '(Condeer.] Petersburg den 18. Dec. Se.russisch- kaiserl.']
['Prinzen', 76, 83, 'pers', ' kaiserl. Majestät haben dem Prinzen vonCondé bey dessen Ankunft in']


['Petersburg', 115, 125, 'LOC', ' bey dessen Ankunft in Petersburg den St.Andreas-Orden u. den Maltheser']
['Petersburg', 115, 125, 'loc', ' bey dessen Ankunft in Petersburg den St.Andreas-Orden u. den Maltheser']


['St', 130, 132, 'LOC', ' Ankunft in Petersburg den St.Andreas-Orden u. den Maltheser Ritterorden']
['Polen', 179, 184, 'loc', ' den Maltheser Ritterorden in Polen zu ertheilen, und ihn mit']


['Polen', 179, 184, 'LOC', ' den Maltheser Ritterorden in Polen zu ertheilen, und ihn mit']
['Petersburg', 256, 266, 'loc', ' prächtigen,völlig meublirten Palais in Petersburg zu beschenken geruhet. Das aus']


['Petersburg', 256, 266, 'LOC', '

In [81]:
def evaluate(res,correction):        
    VP,FP,FN = [],[],[]
    for i in range(len(res)):
        a = res[i]
        b = correction[i]
        i,j = 0,0
        while i < len(a) and j < len(b):
            starta,enda,labela = a[i][1],a[i][2],a[i][3]
            startb,endb,labelb = b[j][1],b[j][2],b[j][3]
            #print(a[i]," ",b[i]," ",(i,j))
            if starta==startb and enda==endb:
                if labela.lower()==labelb.lower():
                    VP.append(a[i])
                else:
                    a[i][3] = labela+" but correct : {}".format(labelb)
                    FP.append(a[i])
                i+=1
                j+=1
            elif enda < startb:
                FP.append(a[i])
                i+=1
            elif endb < starta:
                FN.append(b[j])
                j+=1
            else:
                if labela.lower()==labelb.lower():
                    VP.append(a[i])
                else:
                    a[i][3]  = labela+" but correct : {}".format(labelb)
                    FP.append(a[i])
                i+=1
                j+=1  
            #print("VP : ",len(VP)," FP : ",len(FP)," FN : ",len(FN)," ",(i,j))
        while i < len(a):
            FP.append(a[i])
            i+=1
        while j < len(b):
            FN.append(b[j])
            j+=1
    return [VP,FP,FN]

a = create[2:3]
b = reference[2:3]
l = evaluate(a,b)
print([len(x) for x in l])

[2, 19, 13]


In [79]:
res_create = evaluate(create,reference)
res_sm_train = evaluate(sm_train,reference)
res_md_train = evaluate(md_train,reference)

In [80]:
print([len(x) for x in res_create])
print([len(x) for x in res_sm_train])
print([len(x) for x in res_md_train])

[355, 502, 784]
[377, 406, 742]
[416, 532, 670]


In [82]:
def ecrire(filename,data):
    with open("visualisation_data/{}.txt".format(filename),"w",encoding="utf-8") as f:
        for i in range(len(data)):
            l = "|".join([str(x) for x in data[i]])
            f.write(l)
            if i < len(data)-1:
                f.write("\n")
        
vp_create,fp_create,fn_create = 

In [199]:
result1 = evaluate(res1,correction)
result2 = evaluate(res2,correction)

In [200]:
import pandas as pd

d1 = pd.DataFrame({"NER":[x[0] for x in result1[1]],"Type":[x[3] for x in result1[1]],"Context":[x[4] for x in result1[1]]},index = [x for x in range(1,len(result1[1])+1)])
d2 = pd.DataFrame({"NER":[x[0] for x in result1[2]],"Type":[x[3] for x in result1[2]],"Context":[x[4] for x in result1[2]]},index = [x for x in range(1,len(result1[2])+1)])

d3 = pd.DataFrame({"NER":[x[0] for x in result2[1]],"Type":[x[3] for x in result2[1]],"Context":[x[4] for x in result2[1]]},index = [x for x in range(1,len(result2[1])+1)])
d4 = pd.DataFrame({"NER":[x[0] for x in result2[2]],"Type":[x[3] for x in result2[2]],"Context":[x[4] for x in result2[2]]},index = [x for x in range(1,len(result2[2])+1)])


html_string1 = '''
<html>
  <meta charset="UTF-8"></meta>
  <head><title>Faux Postif</title></head>
  <body>
    {table}
  </body>
</html>.
'''

html_string2 = '''
<html>
  <meta charset="UTF-8"></meta>
  <head><title>Vrai Négatif</title></head>
  <body>
    {table}
  </body>
</html>.
'''

with open("résultat/Model only with train/Faux Positif.html","w",encoding='utf-8') as f:
    f.write(html_string1.format(table=d1.to_html()))
with open("résultat/Model only with train/Vrai Négatif.html","w",encoding='utf-8') as f:
    f.write(html_string2.format(table=d2.to_html()))
with open("résultat/Model with train + Spacy Model/Faux Positif.html","w",encoding='utf-8') as f:
    f.write(html_string1.format(table=d3.to_html()))
with open("résultat/Model with train + Spacy Model/Vrai Négatif.html","w",encoding='utf-8') as f:
    f.write(html_string2.format(table=d4.to_html()))

### Statistics for the results


#### Type of NER

In [201]:
def stats(result2,typeLabel):
    total = []
    trouvé = list(filter(lambda x:x[3]==typeLabel,result2[0]))
    incorrect = list(filter(lambda x:x[3]==typeLabel,result2[1]))
    non_trouvé = list(filter(lambda x:x[3]==typeLabel,result2[2]))
    total+=(trouvé+incorrect+non_trouvé)
    p = round(len(trouvé)/(len(trouvé)+len(incorrect))*100,3)
    r = round(len(trouvé)/(len(non_trouvé)+len(trouvé))*100,3)
    if p+r!=0:
        f = round(2*p*r/(p+r),3)
    else:
        f = 0.0
    return [p,r,f]

##### 1. LOC

In [202]:
loc = stats(result2,"LOC")
print(loc)

[76.432, 55.878, 64.558]


###### 2. PERS

In [203]:
pers = stats(result2,"PER")
print(pers)

[56.727, 40.31, 47.13]


###### 3. Orgs

In [204]:
orgs = stats(result2,"ORG")
print(orgs)

[77.083, 25.17, 37.949]


##### 4. MISC

In [205]:
misc = stats(result2,"MISC")
print(misc)

[61.702, 23.77, 34.319]


In [206]:
import pandas as pd
res = pd.DataFrame({"Précision":[loc[0],pers[0],orgs[0],misc[0]],"Rappel":[loc[1],pers[1],orgs[1],misc[1]],'F-Mesure':[loc[2],pers[2],orgs[2],misc[2]]},index = ["Location","Person","Organisation","Miscellaneous entities"])
res

,Précision,Rappel,F-Mesure
Location,76.432,55.878,64.558
Person,56.727,40.310,47.130
Organisation,77.083,25.170,37.949
Miscellaneous entities,61.702,23.770,34.319


In [207]:
html_string1 = '''
<html>
  <meta charset="UTF-8"></meta>
  <head><title>Faux Postif</title></head>
  <body>
    {table}
  </body>
</html>.
'''

html_string2 = '''
<html>
  <meta charset="UTF-8"></meta>
  <head><title>Vrai Négatif</title></head>
  <body>
    {table}
  </body>
</html>.
'''
with open("résultat/Model with train + Spacy Model/Chaque_type.html","w",encoding='utf-8') as f:
    f.write(html_string1.format(table=res.to_html()))

### Observe every NER not finded or finded but not correct

##### Incorrect

In [208]:
import collections
from collections import Counter
fp = dict(Counter([x[0] for x in result2[1]]))
fp = sorted(fp.items(),key=lambda x:x[1],reverse=True)
print(fp)

[('Zürich', 4), ('von', 3), ('Paris', 3), ('Preußen', 3), ('Nationalrath', 3), ('Baden', 3), ('Bundesrath', 3), ('Sängervereins', 3), ('Schweiz', 3), ('Ferdinand', 3), ('Se', 2), ('Lord', 2), ('vom', 2), ('Asselt', 2), ('am', 2), ('Kompagnien', 2), ('Bucharest', 2), ('Baron', 2), ('Jedermann', 2), ('Nationalgarde', 2), ('Herren', 2), ('seit', 2), ('Papier', 2), ('Sündfluth', 2), ('XVI', 2), ('Noven', 2), ('Graf', 2), ('Bürgerschaft', 2), ('Chöre', 2), ('Händler', 2), ('Basel', 2), ('Nokraschi', 2), ('Ministeriums', 2), ('Condeer', 1), ('St', 1), ('Kowelin', 1), ('Duc', 1), ('Königreichs', 1), ('FortsetzungIhrer', 1), ('Gefahrenund', 1), ('Unglük', 1), ('Sinclair', 1), ('consolidirten3', 1), ('Landein', 1), ('Courier', 1), ('Belgische', 1), ('vereinigten', 1), ('Sittard', 1), ('Lüttich', 1), ('Betagerungsstand', 1), ('Roumiroir', 1), ('Kühnheitgehabt', 1), ('Löwen', 1), ('Batavische', 1), ('Schuldenscheine', 1), ('Sevilla', 1), ('Hauptstraßenach', 1), ('Saragossa', 1), ('Gegenwart', 1),

##### Non_trouvé

In [209]:
import collections
from collections import Counter
fn = dict(Counter([x[0] for x in result2[2]]))
fn = sorted(fn.items(),key=lambda x:x[1],reverse=True)
print(fn)

[('Hr', 22), ('Hrn', 9), ('Zürich', 9), ('Dr', 9), ('von', 8), ('Guyer', 6), ('Kirchgemeinde', 6), ('St', 5), ('Schweiz', 5), ('1937', 5), ('Deutschland', 4), ('Prof', 4), ('im', 4), ('Bürgliterrasse', 4), ('Bundesgericht', 4), ('Küßnacht', 4), ('Volksrecht', 4), ('Prinzen', 3), ('Havre', 3), ('alt', 3), ('Solothurner', 3), ('Oesterreich', 3), ('A', 3), ('L', 3), ('Köln', 3), ('Wafd', 3), ('Nokraschi', 3), ('junge', 2), ('das', 2), ('Gerona', 2), ('Wohlen', 2), ('Bundeszeitung', 2), ('Journal', 2), ('New', 2), ('hannoversche', 2), ('Preußenverein', 2), ('Berlin', 2), ('Winterthur', 2), ('Liestal', 2), ('Basellandschaftliche', 2), ('Wallis', 2), ('Oberrichter', 2), ('Clement', 2), ('Madrid', 2), ('dem', 2), ('Toledo', 2), ('Andrassy', 2), ('Freiburg', 2), ('Lausanner', 2), ('Bundesversammlung', 2), ('Kellersberger', 2), ('KantonsZürich', 2), ('Gobat', 2), ('am', 2), ('Frey', 2), ('Schweizerische', 2), ('Interlaken', 2), ('vom', 2), ('Lauterbrunnen', 2), ('Männerchor', 2), ('Bürgerlichen

In [212]:
import pandas as pd
tf_incorrect = pd.DataFrame({"NER":[x[0] for x in fp],"Frequecy":[x[1] for x in fp]})
tf_non_trouvé = pd.DataFrame({"NER":[x[0] for x in fn],"Frequency":[x[1] for x in fn]})


html_string1 = '''
<html>
  <meta charset="UTF-8"></meta>
  <head><title>Term Frequency for incorrect NER</title></head>
  <body>
    {table}
  </body>
</html>.
'''

html_string2 = '''
<html>
  <meta charset="UTF-8"></meta>
  <head><title>Term Frequency for non correct NER</title></head>
  <body>
    {table}
  </body>
</html>.
'''
with open("résultat/Model with train + Spacy Model/tfFP.html","w",encoding='utf-8') as f:
    f.write(html_string1.format(table=tf_incorrect.to_html()))

with open("résultat/Model with train + Spacy Model/tfFN.html","w",encoding='utf-8') as f:
    f.write(html_string1.format(table=tf_non_trouvé.to_html()))

In [210]:
"Prinzen" in set([x[0] for x in result2[0]])

False

In [211]:
"Zürich" in set([x[0] for x in result2[0]])

True

### Zürich exists both in VP and Faux Negatif

In [214]:
Zürich_vp = list(filter(lambda x:x[0]=="Zürich",result2[0]))
Zürich_fp = list(filter(lambda x:x[0]=="Zürich",result2[1]))
Zürich_fn = list(filter(lambda x:x[0]=="Zürich",result2[2]))
print(Zürich_vp[0],len(Zürich_vp))
print(Zürich_fp[0],len(Zürich_fp))
print(Zürich_fn[0],len(Zürich_fn))

['Zürich', 3, 9, 'LOC', 'In Zürich starb am 2. April Hr.'] 30
['Zürich', 785, 791, 'LOC', ' gesetzt wurden.— Der Stadtverein Zürich hat in seiner Versammlung vom'] 4
['Zürich', 2111, 2117, 'LOC', ' in Zürich, Nationalrath Fierz inZürich, Oberstl. v. Muralt in'] 9


In [216]:
html_string = '''
<html>
  <meta charset="UTF-8"></meta>
  <head><title>Zürich</title></head>
  <body>
    {table}
  </body>
</html>.
'''
tmp1 = [x[4] for x in Zürich_vp]
tmp2 = [x[4] for x in Zürich_fn]
tmp3 = [x[4] for x in Zürich_fp]
a,b,c = len(tmp1),len(tmp2),len(tmp3)
if a<b:
    for x in range(b-a):
        tmp1.append('')
else:
    for x in range(a-b):
        tmp2.append("")
    for x in range(a-c):
        tmp3.append('')
    

l1 = pd.DataFrame({"Vrai Postif":tmp1,"Faux positif":tmp3,"Faux Negatif":tmp2},index = [x for x in range(1,max(a,b)+1)])
with open("résultat/Model with train + Spacy Model/Zürich.html","w",encoding='utf-8') as f:
    f.write(html_string.format(table=l1.to_html()))

In [220]:
import nltk
import sklearn_crfsuite

from copy import deepcopy
from collections import defaultdict

from sklearn_crfsuite import metrics

import NER_Evaluation 
from NER_Evaluation import ner_evaluation



In [232]:
result2[1][0]

['Condeer', 1, 8, 'LOC', '(Condeer.] Petersburg den 18. Dec.']

In [234]:
correction[1][0]

['Petersburg',
 11,
 21,
 'LOC',
 '(Condeer.] Petersburg den 18. Dec. Se.russisch- kaiserl.']

In [235]:
from spacy import displacy

corpus = result2[1][0]
doc = SM_plus_Train_DATA_Model("Rußland. (Condeer.] Petersburg den 18")
displacy.serve(doc,style = "ent")

/Users/zijian/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [244]:
doc = SM_plus_Train_DATA_Model("sagt man, daß die Universität Zürich an")
displacy.serve(doc,style = "ent")

/Users/zijian/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
